In [ ]:
import sys
import os
import time

In [ ]:
# include path to our library
cwd = os.getcwd()
dir_path = os.path.dirname(os.path.realpath(cwd))
sys.path.append(dir_path)
time.sleep(2)

In [ ]:
dir_path
%cd $dir_path
!ls

In [ ]:
import argparse
import json

from sklearn.model_selection import train_test_split

from core.utils.metrics import *
from core.utils.utils import *
from core.data.data_loader import *

In [ ]:
# load some configurations configurations of model and others
configs = json.load(open('config_lstm.json', 'r'))
output_path = configs['paths']['save_dir']
output_logs = configs['paths']['log_dir']

if os.path.isdir(output_path) == False:
    os.mkdir(output_path)

if os.path.isdir(output_logs) == False:
    os.mkdir(output_logs)        

time_steps =  configs['model']['layers'][0]['input_timesteps']  # the number of points or hits
num_features = configs['model']['layers'][0]['input_features']  # the number of features of each hits
split_data = configs['data']['train_split']  # the number of features of each hits
cylindrical = configs['data']['cylindrical']  # set to polar or cartesian coordenates
#cylindrical = False
normalise = configs['data']['normalise'] 
num_hits = configs['data']['num_hits']

print('OK reading of json file')


In [ ]:
# reading path of data
path = configs['data']['filename']
#path = '/data/track-ml/eramia/dataset/phi025-025_eta025-025_train1_lasthit_20200219.csv'
path

In [ ]:
data = Dataset(path, split_data, cylindrical, num_hits, KindNormalization.Zscore)
# divimos 

In [ ]:
X_train, y_train = data.get_training_data(n_hit_in=time_steps, n_hit_out=1,
                                 n_features=num_features, normalise=False)


#X_train = X_train.iloc[:,]
#y_test = y_test[0:1000]
print(X_train.shape)
print(y_train.shape)


In [ ]:
#separamos as layers
layer_1  = data.data.iloc[:, 0:3]
layer_2  = data.data.iloc[:, 3:6]
layer_3  = data.data.iloc[:, 6:9]
layer_4  = data.data.iloc[:, 9:12]
layer_5 = data.data.iloc[:, 12:15]
layer_6 = data.data.iloc[:, 15:18]
layer_7 = data.data.iloc[:, 18:21]
layer_8 = data.data.iloc[:, 21:24]
layer_9 = data.data.iloc[:, 24:27]
layer_10 = data.data.iloc[:, 27:30]

#layer_1  = data.data.iloc[0:5000, 0:3]
#layer_2  = data.data.iloc[0:5000, 3:6]
#layer_3  = data.data.iloc[0:5000, 6:9]
#layer_4  = data.data.iloc[0:5000, 9:12]
#layer_5 = data.data.iloc[0:5000, 12:15]
#layer_6 = data.data.iloc[0:5000, 15:18]
#layer_7 = data.data.iloc[0:5000, 18:21]
#layer_8 = data.data.iloc[0:5000, 21:24]
#layer_9 = data.data.iloc[0:5000, 24:27]
#layer_10 = data.data.iloc[0:5000, 27:30]


In [ ]:
layer_5.describe()

In [ ]:
#existe valores duplicados?
layer_1_ = layer_1[layer_1.duplicated()]
#layer_1_ existe valores repetido
layer_2_ = layer_2[layer_2.duplicated()]
#layer_2_  no existe
layer_3_ = layer_3[layer_3.duplicated()]
#layer_3_ no exisste
layer_4_ = layer_4[layer_4.duplicated()]
#layer_4_ # no existe

layer_5_ = layer_5[layer_5.duplicated()]
layer_6_ = layer_6[layer_6.duplicated()]
#print(layer_6_)
layer_5_


In [ ]:
layer_5_.to_csv('duplicados_layer_5.csv')

In [ ]:
# e1:2356 e2:1643 dist=0.0
# e1:2523 e2:1497 dist=0.0
# e1:2615 e2:1403 dist=0.0
layer_5.iloc[1643,:]


In [ ]:
layer_5.iloc[2356,:]

In [ ]:
# elimino valores duplicados
layer_1_ = layer_1.drop_duplicates()
print('new size :', layer_1_.count())

layer_2_ = layer_2.drop_duplicates()
print('new size :', layer_2_.count())

layer_3_ = layer_3.drop_duplicates()
print('new size :', layer_3_.count())

layer_4_ = layer_4.drop_duplicates()
print('new size :', layer_4_.count())

layer_5_ = layer_5.drop_duplicates()
print('new size :', layer_5_.count())

layer_6_ = layer_6.drop_duplicates()
print('new size :', layer_6_.count())

layer_7_ = layer_7.drop_duplicates()
print('new size :', layer_7_.count())

layer_8_ = layer_8.drop_duplicates()
print('new size :', layer_8_.count())

layer_9_ = layer_9.drop_duplicates()
print('new size :', layer_9_.count())

layer_10_ = layer_10.drop_duplicates()
print('new size :', layer_10_.count())

In [ ]:
#juntamos todas as camadas
sensor = data_true = np.concatenate([layer_1_, layer_2_, layer_3_, layer_4_ ,
                                     layer_5_, layer_6_, layer_7_, layer_8_,
                                     layer_9_, layer_10_], axis = 0)
sensor = pd.DataFrame(sensor, columns=['x','y', 'z'])

sensor.shape

## Calculamos a minima distancia por Camada ##

In [ ]:
# selecionamos que data set se normalizamos ou não
#matrix = StandardScaler().fit_transform(layer_5_)
#dataset = pd.DataFrame(layer_5_, columns=['x','y', 'z'])

matrix = layer_5.as_matrix()

In [ ]:
from scipy.spatial import distance

# Por camada

max_dist = 1000
seed = 0
total = len(matrix)
visited = []
visited.append(seed)
temp = {}
counter = 0
print('total ', total)
print('processing...')
shortest_point = {}
all_distances = []
for i in range(0, total):
   
    point = np.reshape(matrix[i], (1, 3))
    matrix = np.delete(matrix, i, axis=0)
    distances = distance.cdist(matrix, point, 'euclidean')  
    min_dist = np.min(distances)
    idx = np.argmin(distances)
    matrix = np.insert(matrix, i, point[0], axis=0) 
    all_distances.append([np.mean(distances), np.min(distances), np.max(distances)])
    
    if min_dist <= max_dist:
        max_dist = min_dist
        shortest_point[i] = idx #matrix[idx].tolist()

for key, value in shortest_point.items():
    e1 = matrix[key].tolist()
    e2 = matrix[value].tolist()
    print(' e1:%s e2:%s dist=%s' % (key, value, calculate_distances_vec(e1,e2)))
    #print(item)
print(min_dist)
print(shortest_point)

In [ ]:
# imprimimos a media, minima e maxima distancia
sorted(all_distances)
#print(len(all_distances))


In [ ]:
# quaeis sao os hits de menor distancia?
for key, value in shortest_point.items():
    e1 = matrix[key].tolist()
    e2 = matrix[value].tolist()
    print(' e1:%s \n e2:%s' % (e1, e2))


## Encontramos os Clusteres de acordo as distancias anteriores ##

In [ ]:
# selecionamos que data set
#dataset = StandardScaler().fit_transform(layer_5_)
#dataset = pd.DataFrame(dataset, columns=['x','y', 'z'])

layer_5_.columns = ['x','y', 'z']
dataset = layer_5_

In [ ]:
from sklearn.cluster import DBSCAN

#model = DBSCAN(algorithm='auto', eps=0.15, min_samples=5) --layer 5
model = DBSCAN(algorithm='auto', eps=5, min_samples=4)
model.fit(dataset)
#model.fit(layer_5_)
model

In [ ]:
labels = model.labels_
print(labels)
print(set(labels))


In [ ]:
# pontos ruido -1
noise = list(labels).count(-1)
noise

In [ ]:
len(pd.DataFrame(labels, columns=['label']).label.value_counts())

In [ ]:
# numero de clusteres
n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
n_clusters

In [ ]:
cores = np.zeros_like(labels, dtype=bool)
cores
print(len(cores))

In [ ]:
model.core_sample_indices_


In [ ]:
cores[model.core_sample_indices_] = True


In [ ]:
set(labels)

In [ ]:
# plotamos todas as camadas en 2D
fig, ax = plt.subplots(figsize=(10,8))
ax.scatter(sensor.x, sensor.y, s=3)
ax.set_xlabel('X')
ax.set_ylabel('Y')
#plt.title('Detector layers(5000 tracks ~ 49900 hits)')
plt.title('Detector layers')
plt.savefig('detector.pdf')
fig.show()


In [ ]:
unique_labels = set(labels)
colors = [plt.cm.Spectral(each) for each in np.linspace(0, 1, len(unique_labels))]



In [ ]:
# plotamos so uma camada
fig, ax = plt.subplots(figsize=(14,12))
ax.scatter(dataset.x, dataset.y,marker='o', c=labels)
fig.show()

In [ ]:
# plotamos so uma camada
fig, ax = plt.subplots(figsize=(14,12))
ax.scatter(layer_5_.x, layer_5_.y, c=cores)
fig.show()

In [ ]:
# Plotamos os cluster de acordo a configurção
X = layer_5_
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]

fig, ax = plt.subplots(figsize=(14,12))

for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    # all classes
    class_member_mask = (labels == k)

    xy = X[class_member_mask & cores]
    plt.plot(xy.iloc[:, 0], xy.iloc[:, 1], 'o', c=tuple(col),
             markersize=3)

    xy = X[class_member_mask & ~cores]
    plt.plot(xy.iloc[:, 0], xy.iloc[:, 1], 'o', c=tuple(col),
               markersize=6)
#markeredgecolor='k',
plt.title('Estimated number of clusters: %d' % n_clusters)
plt.show()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np

# plotamos todas as camadas em 3d 
fig = go.Figure(data=[go.Scatter3d(x=sensor.x,
                                   y=sensor.y,
                                   z=sensor.z,
                                   mode='markers',
                                   marker=dict(
                                        size=2.5,
                                        #color=labels,                # set color to an array/list of desired values
                                        colorscale='Jet',   # choose a colorscale
                                        opacity=0.8
                                    )
                                  )])


#fig.show()
fig.write_html('sensor_labels.html', auto_open=True)  

In [ ]:
#res = (==5)
#print(len(res))
#print(len(labels))
labels

In [ ]:
#for each in np.linspace(0, 1, len(unique_labels)):
for each in range(0, len(unique_labels)):
    print(each)
    #colors = matplotlib.colors.colorConverter.to_rgb(plt.cm.Spectral(each))
    colors = 'rgb'+str(colors)
    print(colors)

colors = plt.cm.Spectral(labels.astype(float) / n_clusters)
#print(colors)

In [ ]:
# graficamos so por camada
import matplotlib.pyplot as plt
colors  =[]

#matplotlib.colors.colorConverter.to_rgb()
#for each in np.linspace(0, 1, len(unique_labels)):
for each in len(unique_labels):
    
    colors = matplotlib.colors.colorConverter.to_rgb(plt.cm.Spectral(each/n_clusters))
    colors = 'rgb'+str(colors)
    print(colors)
    
X = layer_5_

unique_labels = set(labels)


clusters = []
for k, col in zip(unique_labels, colors):
    print(col)
    print(tuple(col))
    if k == -1:
        # Black used for noise.
        #col = [0, 0, 0, 1]
        col = [1.0, 'rgb(0,0,255)']

    # all classes
    class_member_mask = (labels == k)

    xy = X[class_member_mask & cores]
  
    cluster = go.Scatter3d(x=xy.iloc[:, 0],
                           y=xy.iloc[:, 1],
                           z=xy.iloc[:, 2],
                           mode='markers',
                           marker=dict(
                                size=3,
                                color=col,       # set color to an array/list of desired values
                                #colorscale=col,   # choose a colorscale
                                opacity=0.8 
                            )
                          )
    
    clusters.append(cluster)

    xy = X[class_member_mask & ~cores]

    cluster = go.Scatter3d(x=xy.iloc[:, 0],
                           y=xy.iloc[:, 1],
                           z=xy.iloc[:, 2],
                           mode='markers',
                           marker=dict(
                                size=2,
                                color=tuple(col),       # set color to an array/list of desired values
                                #colorscale=tuple(col),   # choose a colorscale
                                opacity=1 
                            )
                          )
    #clusters.append(cluster)
    
fig = go.Figure(data = clusters)

#fig.show()
fig.write_html('sensor_labels.html', auto_open=True)  

In [ ]:

fig = go.Figure(data=[go.Scatter3d(x=layer_5_.x,
                                   y=layer_5_.y,
                                   z=layer_5_.z,
                                   mode='markers',
                                   marker=dict(
                                        size=2,
                                        color=cores,
                                        # set color to an array/list of desired values
                                        #color='smoker',   # choose a colorscale
                                        colorscale='Jet',
                                        #showscale=True,
                                        #opacity=0.8
                                        #colorscale = [
                                        #    [0., 'rgba(0,0,0, 1)'],
                                        #    [1., 'rgba(255, 225, 255, 1)']]                                       
                                    )
                                  )])


#fig.show()
#fig.write_html('layer_labels.html', auto_open=True)  
fig.write_html('layer_cores.html', auto_open=True)  

In [ ]:

unique_labels = set(labels)
colors = [plt.cm.Spectral(each)for each in np.linspace(0, 1, len(unique_labels))]

fig = go.Figure(data=[go.Scatter3d(x=layer_5_.x,
                                   y=layer_5_.y,
                                   z=layer_5_.z,
                                   mode='markers',
                                   marker=dict(
                                        size=2,
                                        color=labels,
                                        colorscale='Jet',
                                    
                                    )
                                  )])


#fig.show()
#fig.write_html('layer_labels.html', auto_open=True)  
fig.write_html('layer_labels.html', auto_open=True)  

In [ ]:
fig, ax = plt.subplots(figsize=(24,18))
ax.scatter(layer_5.x, layer_5.y , c=sample_cores)
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.title('Outlier Detection')
fig.show()

In [ ]:
fig, ax = plt.subplots(figsize=(12,10))
ax.scatter(X_train.iloc[:,0], X_train.iloc[:,1], c=sample_cores )
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.title('Outlier Detection')
fig.show()